<img src="images/header.png" alt="Logo UCLM-ESII" align="right">

<br><br><br><br>
<h2><font color="#92002A" size=4>Trabajo Fin de Grado</font></h2>

<h1><font color="#6B001F" size=5>Generación automática de playlist de canciones <br> mediante técnicas de minería de datos</font></h1>
<h2><font color="#92002A" size=3>Parte 12 - Resultados</font></h2>

<br>
<div style="text-align: right">
    <font color="#B20033" size=3><strong>Autor</strong>: <em>Miguel Ángel Cantero Víllora</em></font><br>
    <br>
    <font color="#B20033" size=3><strong>Directores</strong>: <em>José Antonio Gámez Martín</em></font><br>
    <font color="#B20033" size=3><em>Juan Ángel Aledo Sánchez</em></font><br>
    <br>
<font color="#B20033" size=3>Grado en Ingeniería Informática</font><br>
<font color="#B20033" size=2>Escuela Superior de Ingeniería Informática | Universidad de Castilla-La Mancha</font>

</div>

---

<br>


<a id="indice"></a>
<h2><font color="#92002A" size=5>Índice</font></h2>

<br>

* [1. Introducción](#section1)
* [2. Definicion de funciones auxiliares](#section2)
* [3. Resultados](#section3)

<br>

---

In [1]:
# Permite establecer la anchura de la celda
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
import os
import numpy as np
import joblib
import json
import pickle
import pandas as pd

from lightfm import LightFM
from tqdm.notebook import tqdm as tqdm_nb
from zipfile import ZipFile

c:\pythonenv\mcidaen\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [3]:
# Variables globales
NUM_THREADS = 8
SEED = 0

# Directorio empleado para guardar/leer los datos generados
DATA_PATH = 'MPD_CSV'
TEST_DATA_PATH = 'MPD_TEST'
MODELS_PATH = "models"
BACKUP_PATH = 'backup'

MODEL_PATH = os.path.join(MODELS_PATH,'lightfm_model.pkl')
MPD_LFM_FILE = os.path.join(DATA_PATH, "mpd_lightfm.pickle")

In [4]:
with open(MPD_LFM_FILE, "rb") as read_file:
    mpd_lfm_dict = pickle.load(read_file)
    
tfg_model = joblib.load(MODEL_PATH)

In [5]:
df_tracks = pd.read_csv(os.path.join(DATA_PATH,'mpd.tracks.csv'))
df_plstrs = pd.read_csv(os.path.join(DATA_PATH,'mpd.pls-tracks.csv'), index_col=0)

# Creamos un diccionario para traducir los id de Spotify
# a nuestro identificador PID
tracks_ids = df_tracks.track_id.to_list()
tracks_translate_dict = dict()
for track_pid, track_id in enumerate(tracks_ids):
    tracks_translate_dict[track_id] = track_pid   

c:\pythonenv\mcidaen\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


---

<br>


<a id="section1"></a>
## <font color="#92002A">1 - Introducción</font>
<br>

Por último, vamos a emplear el conjunto de 10.000 playlists cuyo contenido esta incompleto, y han sido agrupadas en 10 categorías de 1.000 listas, para ver el rendimiento de nuestro modelo ante resultados desconocidos.

En el caso de las métricas de los resultados, volvemos a emplear la precisión cuando k es igual a 10, 20, 50 y 100.

<br>

Antes de ver los resultados obtenidos, vamos a recordar qué categorías tenemos en el conjunto de prueba empleado:

1.	Predicción de pistas para una playlist dado sólo su título.
2.	Predicción de pistas para una playlist dado su título y la primera pista.
3.	Predicción de pistas para una playlist dado su título y las primeras 5 pistas.
4.	Predicción de pistas para una playlist dadas las primeras 5 pistas (sin título).
5.	Predicción de pistas para una playlist dado su título y las primeras 10 pistas.
6.	Predicción de pistas para una playlist dadas las primeras 10 pistas (sin título).
7.	Predicción de pistas para una playlist dado su título y las primeras 25 pistas.
8.	Predicción de pistas para una playlist dado su título y 25 pistas aleatorias.
9.	Predicción de pistas para una playlist dado su título y las primeras 100 pistas.
10.	Predicción de pistas para una playlist dado su título y 100 pistas aleatorias.


<br>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section2"></a>
## <font color="#92002A">2 - Definicion de funciones auxiliares</font>

<br>


<br>


In [6]:
# Calcula las métricas para una playlists
def plstrs_rec_analysis(rec_items, val_items):
    """
    :param rec_items: Items que ha recomendado el modelo.
    :param val_items: Items del conjunto de validación.
    :return: Diccionario con las métricas.
    """
    val_set = set(val_items)
    rec10_set = set(rec_items[:10])
    rec20_set = set(rec_items[:20])
    rec50_set = set(rec_items[:50])
    rec100_set = set(rec_items[:100])
    
    analysis_results = dict()
    
    analysis_results['top_10'] = {'common' : rec10_set.intersection(val_set),
                                  'score' : len(rec10_set.intersection(val_set))/len(val_set)}
    analysis_results['top_20'] = {'common' : rec20_set.intersection(val_set),
                                  'score' : len(rec20_set.intersection(val_set))/len(val_set)}
    analysis_results['top_50'] = {'common' : rec50_set.intersection(val_set),
                                  'score' : len(rec50_set.intersection(val_set))/len(val_set)}
    analysis_results['top_100'] = {'common' : rec100_set.intersection(val_set),
                                   'score' : len(rec100_set.intersection(val_set))/len(val_set)}

    return analysis_results

In [7]:
# Realiza la predicción para una playlists del conjunto de prueba
def make_test_prediction(model, data, pls_pids, u_features=None, N=10):
    """
    :param model: Modelo a emplear.
    :param data: Matriz de interacciones (Sparse Matrix).
    :param pls_pids: Usuarios de los que se desea obtener recomendaciones.
    :param labels : Etiquetas/nombres de los items
    :param N: (Opcional) Número de items a predecir similares, por defecto se devuelven 10.
    :return: Lista de diccionarios.
    """
    results_list = []

    #Numero de usuarios e items de la matriz
    n_pls, n_items = data.shape

    #Genera recomendación para cada usuario
    for pl_id in tqdm_nb(pls_pids):
        
        #Items que el usuario conoce
        known_positives = data.tocsr()[pl_id].indices
        
        #Items que nuestro modelo predice al usuario
        scores = model.predict(pl_id, np.arange(n_items), user_features=u_features, num_threads=NUM_THREADS)
        #Ordena los items por puntuación
        top_items = np.argsort(-scores)

        # Borramos los items conocidos de las recomendaciones
        sorter = np.argsort(top_items)
        removable_indices = sorter[np.searchsorted(top_items, known_positives, sorter=sorter)]
        top_items = np.delete(top_items,removable_indices)

        # Resultados
        result_dict = {'pl_pid': pl_id,
                       'known_items' : list(known_positives),
                       'recommended_items' : list(top_items[:N])}
        
        results_list.append(result_dict)
        
    return results_list

In [8]:
def get_category_metrics(rec_pls, val_pls, n_categories, num_pls_category):
    results_dict = dict()
    start_pid = 0
    end_pid = num_pls_category
    
    
    for i in range(0,n_categories):
        res_category = []
        for j in range(start_pid,end_pid):
            res_category.append(plstrs_rec_analysis(rec_pls[j]['recommended_items'],
                                                    val_pls[j]['validation_tracks']))
            
        results_dict[f"Category_{i+1}"] = res_category
        
        start_pid += num_pls_category
        end_pid += num_pls_category
        
    return results_dict

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<a id="section3"></a>
## <font color="#92002A">3 - Resultados</font>

<br>


<br>


In [9]:
predictions_file = os.path.join(BACKUP_PATH,'pls_test_predictions.pkl')

if os.path.isfile(predictions_file):
    with open(predictions_file, "rb") as read_file:
        test_set_predictions = pickle.load(read_file)
else:
    test_set_predictions = make_test_prediction(tfg_model, mpd_lfm_dict['plstrs_interactions'], 
                                                range(1000000,1010000),
                                                u_features=mpd_lfm_dict['pls_features'], N=500)
    with open(predictions_file, "wb") as write_file:
        pickle.dump(test_set_predictions, write_file, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
results_file = os.path.join(BACKUP_PATH,'pls_test_results.pkl')

if os.path.isfile(results_file):
    with open(results_file, "rb") as read_file:
        test_set_results = pickle.load(read_file)
else:
    # Leemos el fichero JSON comprimido con las soluciones
    mpd_test_results_file = os.path.join(TEST_DATA_PATH,'mpd.test_complete.zip')
    with ZipFile(mpd_test_results_file,'r') as zip_file:
        with zip_file.open(zip_file.namelist()[0]) as json_file:
            data = json_file.read()
            data = json.loads(data.decode())
    
    test_set_results = []
    for pl in tqdm_nb(data['playlists']):
        r = dict()
        r['pid'] = pl['pid']
        r['name'] = pl['name']
        tracks_list = []
        for track in pl['tracks']:
            tr_id = track['track_uri'].split(':')[-1]        
            tracks_list.append(tracks_translate_dict[tr_id])
        known_tracks = df_plstrs[df_plstrs.index == pl['pid']].track_pid.to_list()
        for track in known_tracks : tracks_list.remove(track)
        r['known_tracks'] = known_tracks
        r['validation_tracks'] = tracks_list
        test_set_results.append(r)
        
    with open(results_file, "wb") as write_file:
        pickle.dump(test_set_results, write_file, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
results = get_category_metrics(test_set_predictions, test_set_results, 10, 1000)

In [12]:
k = 100
list_results = []
for pos in range(9000,10000):
    ret = set(test_set_predictions[pos]['recommended_items'][:k])
    rel = set(test_set_results[pos]['validation_tracks'])
    result = len(ret.intersection(rel))/min(len(ret),len(rel))
    list_results.append(result)

In [13]:
for category in results.items():
    for category_key in category:
        print(category_key)
        print('-' * 12)
        
        top_10_scores = []
        top_20_scores = []
        top_50_scores = []
        top_100_scores = []
        
        for r in results[category_key]:
            top_10_scores.append(r['top_10']['score'])
            top_20_scores.append(r['top_20']['score'])
            top_50_scores.append(r['top_50']['score'])
            top_100_scores.append(r['top_100']['score'])
            
        print('precision@10:\t',sum(top_10_scores)/1000)
        print('precision@20:\t',sum(top_20_scores)/1000)
        print('precision@50:\t',sum(top_50_scores)/1000)
        print('precision@100:\t',sum(top_100_scores)/1000)
        
        break
    print("")

Category_1
------------
precision@10:	 0.01154549907255993
precision@20:	 0.02021793233261168
precision@50:	 0.039427970885525285
precision@100:	 0.06130557730607572

Category_2
------------
precision@10:	 0.025071236604546417
precision@20:	 0.042084862463926155
precision@50:	 0.08015292991214694
precision@100:	 0.11921548263613857

Category_3
------------
precision@10:	 0.03796103436348459
precision@20:	 0.06140177505199794
precision@50:	 0.11480495012300322
precision@100:	 0.17381157943718004

Category_4
------------
precision@10:	 0.03655898939747491
precision@20:	 0.062245915208051825
precision@50:	 0.11617549449005804
precision@100:	 0.17463617582472074

Category_5
------------
precision@10:	 0.04099674446246121
precision@20:	 0.06771235437878595
precision@50:	 0.13185734919031733
precision@100:	 0.19636569688854563

Category_6
------------
precision@10:	 0.037361537865082485
precision@20:	 0.06778326820048224
precision@50:	 0.12828542922755803
precision@100:	 0.18908126391685098


Como podemos ver en la tabla anterior, comparando los casos donde se ofrece el mismo número de pistas con título y sin el, categorías 3, 4, 5 y 6, vemos que los resultados de nuestro modelo mejoran sensiblemente si se tiene en cuenta el título.

En las categorías 7, 8, 9 y 10, nuestro modelo es capaz de ofrecer mejores recomendaciones en los casos que se le proporcionan pistas de posiciones aleatorias. Esto se debe a que, en numerosas playlists, las pistas que contienen están agrupadas por artista. Al obtener pistas de esta forma, se proporciona un número de artistas mayor que si recomendásemos las primeras *n* posiciones. 


<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#92002A"></i></font></a>
</div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-graduation-cap" aria-hidden="true" style="color:#92002A"></i> </font></div>